## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


In [1]:
!pip install pyconll

In [2]:
import numpy as np
import pandas as pd
import pyconll
import nltk

from nltk.corpus import brown
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

## загрузка данных

In [3]:
!mkdir datasets

mkdir: cannot create directory ‘datasets’: File exists


In [4]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-09-21 20:43:37--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77.28M   166MB/s    in 0.5s    

2021-09-21 20:43:38 (166 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2021-09-21 20:43:38--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [5]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')
df_accuracy  = pd.DataFrame(columns=['accuracy'])

In [6]:
train_data = [[(token.form, token.upos) for token in sent] for sent in full_train]
test_data = [[(token.form, token.upos) for token in sent] for sent in full_test]

test_sent = [[form for form, _ in row] for row in test_data]


In [7]:
test_sent_choice = np.random.choice(test_sent)
test_sent_choice

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


['Нужны', 'новые', 'станки', ',', 'нужны', 'новые', 'технологии', '.']

In [8]:
print(f'train_data: \n{train_data[:2]}\ntest_data:\n{test_data[:2]}')

train_data: 
[[('Анкета', 'NOUN'), ('.', 'PUNCT')], [('Начальник', 'NOUN'), ('областного', 'ADJ'), ('управления', 'NOUN'), ('связи', 'NOUN'), ('Семен', 'PROPN'), ('Еремеевич', 'PROPN'), ('был', 'AUX'), ('человек', 'NOUN'), ('простой', 'ADJ'), (',', 'PUNCT'), ('приходил', 'VERB'), ('на', 'ADP'), ('работу', 'NOUN'), ('всегда', 'ADV'), ('вовремя', 'ADV'), (',', 'PUNCT'), ('здоровался', 'VERB'), ('с', 'ADP'), ('секретаршей', 'NOUN'), ('за', 'ADP'), ('руку', 'NOUN'), ('и', 'CCONJ'), ('иногда', 'ADV'), ('даже', 'PART'), ('писал', 'VERB'), ('в', 'ADP'), ('стенгазету', 'NOUN'), ('заметки', 'NOUN'), ('под', 'ADP'), ('псевдонимом', 'NOUN'), ('"', 'PUNCT'), ('Муха', 'NOUN'), ('"', 'PUNCT'), ('.', 'PUNCT')]]
test_data:
[[('Алгоритм', 'NOUN'), (',', 'PUNCT'), ('от', 'ADP'), ('имени', 'NOUN'), ('учёного', 'NOUN'), ('аль', 'PART'), ('-', 'PUNCT'), ('Хорезми', 'PROPN'), (',', 'PUNCT'), ('-', 'PUNCT'), ('точный', 'ADJ'), ('набор', 'NOUN'), ('инструкций', 'NOUN'), (',', 'PUNCT'), ('описывающих', 'VERB')

## 1.1

In [9]:
taggers = [UnigramTagger, BigramTagger, TrigramTagger]

for tag_c in taggers:
  tagger = tag_c(train_data)
  print(tag_c.__name__)
  display(tagger.tag(test_sent_choice), tagger.evaluate(test_data))
  df_accuracy.loc[tag_c.__name__, 'accuracy'] = tagger.evaluate(train_data)

UnigramTagger


[('Нужны', 'ADJ'),
 ('новые', 'ADJ'),
 ('станки', 'NOUN'),
 (',', 'PUNCT'),
 ('нужны', 'ADJ'),
 ('новые', 'ADJ'),
 ('технологии', 'NOUN'),
 ('.', 'PUNCT')]

0.8772537323492737

BigramTagger


[('Нужны', 'ADJ'),
 ('новые', 'ADJ'),
 ('станки', 'NOUN'),
 (',', 'PUNCT'),
 ('нужны', 'ADJ'),
 ('новые', 'ADJ'),
 ('технологии', 'NOUN'),
 ('.', 'PUNCT')]

0.6963064064974893

TrigramTagger


[('Нужны', 'ADJ'),
 ('новые', None),
 ('станки', None),
 (',', None),
 ('нужны', None),
 ('новые', None),
 ('технологии', None),
 ('.', None)]

0.24808748694099012

In [10]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN') 
tag = backoff_tagger(train_data,  
                     taggers,  
                     backoff = backoff) 

df_accuracy.loc['BackoffTagger', 'accuracy'] = tag.evaluate(test_data)
tag.evaluate(test_data) 

0.9119991237825633

## 1.2

In [11]:
%%time
train_tok = [form for row in train_data  for form, _ in row]
train_label = ['NO_TAG' if upos is None else upos for row in train_data for _, upos in row]

test_tok = [form for row in test_data for form, _ in row ]
test_label = ['NO_TAG' if upos is None else upos for row in test_data for _, upos in row]

CPU times: user 343 ms, sys: 993 µs, total: 344 ms
Wall time: 345 ms


In [12]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [13]:
HashingVectorizer.__name__

'HashingVectorizer'

In [19]:
def get_accuracy_vec(vectorizer, train_t, test_t, train_l, test_l):
  
  le = LabelEncoder()
  train_enc_l = le.fit_transform(train_l)
  test_enc_l = le.transform(test_l)

  X_train = vectorizer.fit_transform(train_t)
  X_test = vectorizer.transform(test_t)
  
  print(f'{X_train.shape}\n')

  lr = LogisticRegression(random_state=0)
  lr.fit(X_train, train_enc_l)
  pred = lr.predict(X_test)
  # print(vectorizer.__name__)
  print(vectorizer.get_params())
  if str(vectorizer).split('(')[0] == 'HashingVectorizer':
    vec_name = f"{str(vectorizer).split('(')[0]}: [analyzer:{vectorizer.get_params()['analyzer']}, n_features:{vectorizer.get_params()['n_features']}]"
  else: vec_name = f"{str(vectorizer).split('(')[0]}: [analyzer:{vectorizer.get_params()['analyzer']}]"
  df_accuracy.loc[vec_name, 'accuracy'] = accuracy_score(test_enc_l, pred)
  return f'Accuracy: {accuracy_score(test_enc_l, pred)}'


In [20]:
vectorizers = [HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=100),
               HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=1000),
               HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=1000),
               CountVectorizer(ngram_range=(1, 5), analyzer='char'),  #, binary=False, tokenizer=str.split),
               TfidfVectorizer(ngram_range=(1, 5), analyzer='char'),  #, binary=False, tokenizer=str.split),
               ]

In [21]:
for vectorizer in vectorizers:
  print(get_accuracy_vec(vectorizer=vectorizer,
                         train_t=train_tok,
                         test_t=test_tok,
                         train_l=train_label,
                         test_l=test_label))

(871526, 100)



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'alternate_sign': True, 'analyzer': 'char', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'n_features': 100, 'ngram_range': (1, 5), 'norm': 'l2', 'preprocessor': None, 'stop_words': None, 'strip_accents': None, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None}
Accuracy: 0.686221480807468
(871526, 1000)



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'alternate_sign': True, 'analyzer': 'char', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'n_features': 1000, 'ngram_range': (1, 5), 'norm': 'l2', 'preprocessor': None, 'stop_words': None, 'strip_accents': None, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None}
Accuracy: 0.8805058470663566
(871526, 1000)



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'alternate_sign': True, 'analyzer': 'word', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'n_features': 1000, 'ngram_range': (1, 5), 'norm': 'l2', 'preprocessor': None, 'stop_words': None, 'strip_accents': None, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None}
Accuracy: 0.3789303407137802
(871526, 149809)



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'analyzer': 'char', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.int64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 1, 'ngram_range': (1, 5), 'preprocessor': None, 'stop_words': None, 'strip_accents': None, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'vocabulary': None}
Accuracy: 0.949979779597614
(871526, 149809)

{'analyzer': 'char', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 1, 'ngram_range': (1, 5), 'norm': 'l2', 'preprocessor': None, 'smooth_idf': True, 'stop_words': None, 'strip_accents': None, 'sublinear_tf': False, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'use_idf': True, 'vocabulary': None}
Accuracy: 0.9382266707107472


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## 1.3

In [22]:
df_accuracy

,accuracy
UnigramTagger,0.974003
BigramTagger,0.970001
TrigramTagger,0.894157
BackoffTagger,0.911999
"HashingVectorizer: [analyzer:char, n_features:100]",0.686221
"HashingVectorizer: [analyzer:char, n_features:1000]",0.880506
"HashingVectorizer: [analyzer:word, n_features:1000]",0.37893
CountVectorizer: [analyzer:char],0.94998
TfidfVectorizer: [analyzer:char],0.938227


Прирос при равных параметрах у HashingVect при увеличении количеста фичей.
При внесении тех же данных при смене анализатора падение точности.

Небольшой выигрыш CountVectorizer у TfidfVectorizer

# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
* передаём в сетку токен и его соседей
* передаём в сетку только токен

4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [ ]:
!pip install corus

In [ ]:
import corus

In [ ]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

In [ ]:
!unzip collection5.zip

In [ ]:
!rm collection5.zip

In [ ]:
!ls

In [ ]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)


процедуры обработки взять из вебинарного ноутбука

In [ ]:
# установка deeppavlov

!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

In [ ]:
#!python -m deeppavlov install squad_bert
#!python -m deeppavlov install ner_ontonotes

In [ ]:
import deeppavlov
from deeppavlov import configs, build_model

In [ ]:
deeppavlov_ner = build_model(configs.ner.ner_bert_ent_and_type_rus, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])